In [ ]:
####### VERSÃO 1.1 ###################

In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import pytz
import dash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import base64
import io

# --- Função para Obter e Processar Dados de DPA ---
def get_dpa_data(ticker_symbol: str, anos: int) -> tuple:
    """
    Busca os dados anuais de DPA para um ticker e calcula a média.
    Retorna uma tupla contendo o DataFrame de DPA e a média.
    """
    fuso_horario = pytz.timezone('America/Sao_Paulo')
    end_date = datetime.now(fuso_horario)
    start_date = end_date - timedelta(days=anos * 365)
    
    try:
        ticker = yf.Ticker(ticker_symbol)
        
        if not ticker.history(period="1d").empty:
            dividends = ticker.dividends
            
            if dividends.empty:
                return pd.DataFrame(), 0, f"Sem dados de dividendos para {ticker_symbol}."

            dividends_filtered = dividends.loc[start_date:end_date]
            
            if dividends_filtered.empty:
                return pd.DataFrame(), 0, f"Sem dados de dividendos para {ticker_symbol} nos últimos {anos} anos."

            dividends_by_year = dividends_filtered.resample('YE').sum().reset_index()
            dividends_by_year.columns = ['Ano', 'DPA (R$)']
            dividends_by_year['Ano'] = dividends_by_year['Ano'].dt.year
            dividends_by_year['Ticker'] = ticker_symbol
            
            media_dpa = dividends_by_year['DPA (R$)'].mean()
            
            mensagem = ""
            if len(dividends_by_year) < anos:
                mensagem = f"Aviso: Média de {ticker_symbol} calculada com {len(dividends_by_year)} anos."
            
            return dividends_by_year, media_dpa, mensagem
        
        else:
            return pd.DataFrame(), 0, f"O ticker '{ticker_symbol}' não é válido ou não foi encontrado."
            
    except Exception as e:
        return pd.DataFrame(), 0, f"Ocorreu um erro ao processar o ticker '{ticker_symbol}': {e}"

# --- Função para obter os indicadores fundamentalistas ---
def get_fundamental_data(ticker_symbol: str) -> dict:
    """
    Busca os indicadores fundamentalistas de um ticker.
    Retorna um dicionário com os valores formatados.
    """
    try:
        ticker = yf.Ticker(ticker_symbol)
        info = ticker.info
        
        # Obter o P/L
        pe_ratio = info.get('trailingPE')
        roe = info.get('returnOnEquity')
        roic = info.get('returnOnInvestedCapital')
        dy = info.get('dividendYield')
        debt_equity = info.get('debtToEquity')

        # Formata os valores para exibição
        pe_ratio = f"{pe_ratio:.2f}" if isinstance(pe_ratio, (float, int)) else 'N/A'
        roe = f"{roe:.2%}" if isinstance(roe, (float, int)) else 'N/A'
        roic = f"{roic:.2%}" if isinstance(roic, (float, int)) else 'N/A'
        
        if isinstance(dy, (float, int)):
            if dy > 1:
                dy = f"{dy:.2f}%"
            else:
                dy = f"{dy:.2%}"
        else:
            dy = 'N/A'
        
        if isinstance(debt_equity, (float, int)):
            debt_equity = f"{(debt_equity / 100):.2f}"
        else:
            debt_equity = 'N/A'

        return {
            'Ticker': ticker_symbol,
            'P/L': pe_ratio,
            'ROE': roe,
            'ROIC': roic,
            'DY': dy,
            'Dívida/PL': debt_equity
        }

    except Exception:
        return {
            'Ticker': ticker_symbol,
            'P/L': 'Erro',
            'ROE': 'Erro',
            'ROIC': 'Erro',
            'DY': 'Erro',
            'Dívida/PL': 'Erro'
        }

# --- Configuração do aplicativo Dash ---
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([
    html.H1("Análise de DPA e Indicadores Fundamentalistas", className="text-center my-4"),
    html.P("Escolha uma das opções para inserir os tickers.", className="lead text-center"),
    
    dbc.Row([
        dbc.Col(
            dbc.InputGroup(
                [
                    dbc.InputGroupText("Tickers (separados por vírgula)", style={'background-color': '#e0e0ff'}),
                    dcc.Input(id="ticker-input", type="text", value="PETR4.SA,VALE3.SA", className="form-control", style={'background-color': '#f0f0ff'}),
                ]
            ),
            width=6,
        ),
        dbc.Col(
            dbc.InputGroup(
                [
                    dbc.InputGroupText("Anos", style={'background-color': '#ffe0e0'}),
                    dcc.Input(id="anos-input", type="number", value=5, min=1, className="form-control", style={'background-color': '#fff0f0'}),
                ]
            ),
            width=4,
        ),
        dbc.Col(
            dbc.Button("Gerar Relatório", id="submit-button", n_clicks=0, color="primary", className="w-100"),
            width=2,
        )
    ], className="mb-4"),
    
    html.Hr(),
    
    # --- Seção de Upload com instruções ---
    dbc.Row([
        dbc.Col(
            html.Div([
                html.P("Ou faça o upload de um arquivo CSV com seus tickers."),
                html.P("O arquivo deve ter uma única coluna com o cabeçalho 'Ticker'.", className="text-muted small"),
                dcc.Upload(
                    id='upload-data',
                    children=html.Div([
                        'Arraste e solte ou ',
                        html.A('selecione um arquivo')
                    ]),
                    style={
                        'width': '100%',
                        'height': '60px',
                        'lineHeight': '60px',
                        'borderWidth': '1px',
                        'borderStyle': 'dashed',
                        'borderRadius': '5px',
                        'textAlign': 'center',
                        'margin': '10px'
                    },
                    multiple=False
                ),
            ]),
            width=12
        ),
    ]),
    
    html.Div(id="upload-status", className="text-center"),
    html.Hr(),

    # --- Seção de Explicações dos Indicadores ---
    html.H4("O que significam os indicadores?", className="mt-4"),
    dbc.Card(
        dbc.CardBody([
            html.P([
                html.Strong("P/L (Preço/Lucro)"),
                ": Indica quantos anos de lucro a empresa levaria para gerar o seu valor de mercado atual. Um P/L mais baixo pode sugerir que a ação está subvalorizada."
            ]),
            html.P([
                html.Strong("ROE (Return on Equity)"),
                ": Mostra o quanto a empresa consegue gerar de lucro para cada R$ 1 de patrimônio líquido. Um valor mais alto indica uma gestão mais eficiente."
            ]),
            html.P([
                html.Strong("ROIC (Return on Invested Capital)"),
                ": Mede a eficiência do capital total (dívidas + patrimônio) na geração de lucro. O ideal é que o ROIC seja superior ao Custo Médio Ponderado de Capital (WACC), que é o custo total do dinheiro da empresa. Se o ROIC for maior que o WACC, a empresa está criando valor."
            ]),
            html.P([
                html.Strong("DY (Dividend Yield)"),
                ": É o rendimento de dividendos de uma ação, ou seja, o percentual de retorno que você recebe em dividendos em relação ao preço da ação. Um DY consistente é atrativo para investidores que buscam renda."
            ]),
            html.P([
                html.Strong("Dívida/PL (Dívida/Patrimônio Líquido)"),
                ": Avalia o nível de endividamento da empresa. Um valor menor indica que a empresa tem uma menor proporção de dívidas em relação ao seu capital próprio, o que é geralmente visto como um sinal de solidez financeira."
            ]),
        ]),
        className="mb-4"
    ),

    html.Hr(),
    
    html.Div(id="tabela-output", className="mt-4"),
])

# Callback para processar o upload do arquivo
@app.callback(
    Output('ticker-input', 'value'),
    Output('upload-status', 'children'),
    Input('upload-data', 'contents'),
    State('upload-data', 'filename')
)
def update_tickers_from_upload(contents, filename):
    if contents:
        try:
            content_type, content_string = contents.split(',')
            decoded = base64.b64decode(content_string)
            
            if 'csv' in filename:
                df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
            else:
                return dash.no_update, html.P("Formato de arquivo não suportado. Por favor, use um arquivo CSV.", className="alert alert-danger")

            if 'Ticker' not in df.columns:
                return dash.no_update, html.P("Arquivo CSV deve ter uma coluna chamada 'Ticker'.", className="alert alert-danger")
            
            tickers = ','.join(df['Ticker'].tolist())
            return tickers, html.P(f"Arquivo '{filename}' processado com sucesso! Tickers carregados no campo acima.", className="alert alert-success")

        except Exception as e:
            return dash.no_update, html.P(f"Ocorreu um erro ao processar o arquivo: {e}", className="alert alert-danger")
    
    return dash.no_update, ""

# Callback principal para gerar o relatório
@app.callback(
    Output("tabela-output", "children"),
    [Input("submit-button", "n_clicks")],
    [
        State("ticker-input", "value"),
        State("anos-input", "value")
    ],
    prevent_initial_call=True
)
def update_table(n_clicks, ticker_str, anos):
    if not ticker_str or not anos:
        return html.P("Por favor, preencha todos os campos ou faça o upload de um arquivo.", className="alert alert-warning")
    
    tickers = [t.strip().upper() for t in ticker_str.split(',')]
    all_dfs = []
    media_data = []
    fundamental_data = []
    messages = []

    for ticker in tickers:
        df_dpa, media, mensagem = get_dpa_data(ticker, anos)
        fundamental_dict = get_fundamental_data(ticker)
        
        fundamental_data.append(fundamental_dict)
        
        if not df_dpa.empty:
            all_dfs.append(df_dpa)
            media_data.append({'Ticker': ticker, f'Média DPA (R$ - {anos} anos)': round(media, 2)})
            if "Aviso" in mensagem:
                messages.append(html.P(mensagem, className="text-danger"))
        else:
            messages.append(html.P(mensagem, className="text-danger"))
    
    if not all_dfs:
        return html.P("Nenhum dado foi encontrado para os tickers fornecidos. Verifique a grafia.", className="alert alert-danger")
    
    fundamental_df = pd.DataFrame(fundamental_data)
    fundamental_table = [
        dash_table.DataTable(
            id='fundamental-table',
            columns=[{"name": i, "id": i} for i in fundamental_df.columns],
            data=fundamental_df.to_dict('records'),
            style_header={'backgroundColor': '#e0f7fa', 'fontWeight': 'bold'},
            style_cell={'textAlign': 'center'},
            style_data_conditional=[{'if': {'row_index': 'odd'}, 'backgroundColor': '#f0fdff'}]
        ),
        html.P(
            "Nota: A disponibilidade dos dados fundamentalistas (como ROIC) pode variar dependendo do ticker e da fonte (Yahoo Finance).",
            className="text-muted small mt-2"
        )
    ]

    media_df = pd.DataFrame(media_data)
    media_table = dash_table.DataTable(
        id='media-table',
        columns=[{"name": i, "id": i} for i in media_df.columns],
        data=media_df.to_dict('records'),
        style_header={'backgroundColor': '#f0f4f8', 'fontWeight': 'bold'},
        style_cell={'textAlign': 'center'},
        style_data_conditional=[{'if': {'row_index': 'odd'}, 'backgroundColor': '#f8fafc'}]
    )
    
    final_df = pd.concat(all_dfs)
    
    fig = go.Figure()
    for ticker in final_df['Ticker'].unique():
        df_ticker = final_df[final_df['Ticker'] == ticker]
        fig.add_trace(go.Bar(
            x=df_ticker['Ano'], 
            y=df_ticker['DPA (R$)'], 
            name=ticker
        ))
    fig.update_layout(
        barmode='group',
        title_text='DPA Anual por Ticker',
        xaxis_title="Ano",
        yaxis_title="DPA (R$)",
        template="plotly_white"
    )

    dpa_table = dash_table.DataTable(
        id='dpa-table',
        columns=[
            {"name": i, "id": i} for i in final_df.columns
        ],
        data=final_df.to_dict('records'),
        sort_action="native",
        style_header={'backgroundColor': '#e3f2fd', 'fontWeight': 'bold'},
        style_cell={'textAlign': 'left'},
        style_data_conditional=[
            {'if': {'row_index': 'odd'}, 'backgroundColor': '#f1f8ff'}
        ]
    )
    
    return [
        html.H3("Indicadores Fundamentalistas:", className="mt-4"),
        html.Div(fundamental_table),
        html.H3(f"Média do DPA por Empresa (últimos {anos} anos):", className="mt-4"),
        media_table,
        html.Hr(),
        html.Div(messages),
        html.H3("DPA Anual por Ticker:", className="mt-4"),
        dcc.Graph(figure=fig),
        html.H3("Detalhes do DPA Anual:", className="mt-4"),
        dpa_table
    ]

if __name__ == "__main__":
    app.run(debug=True, port=8051)

In [2]:
!pip freeze > requirements.txt

In [5]:
!echo "web: gunicorn --bind 0.0.0.0 --timeout 600 painel2:server" > Procfile